In [ ]:
import gym
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
env = gym.make('CartPole-v1')
env.reset()

## Defining TRPO setup

In [ ]:
def var_size(v):
    return int(np.prod([int(d) for d in v.shape]))

def get_padded_gradients(loss, var_list):
    grads = tf.gradients(loss, var_list)
    return [g if g is not None else tf.zeros(v.shape)
            for g, v in zip(grads, var_list)]

def get_flattened_gradients(loss, var_list):
    padded_gradients = get_padded_gradients(loss, var_list)
    return tf.concat([tf.reshape(x, [-1]) for x in padded_gradients], 0)

def unflatten_gradient(grad, var_list):
    shapes = [v.shape for v in var_list]
    sizes = [var_size(v) for v in var_list]
    grads = []

    pointer = 0
    for (shape, size, v) in zip(shapes, sizes, var_list):
        grads.append(tf.reshape(grad[pointer:pointer + size], shape))
        pointer += size
    return grads

def sum_discounted_rewards(rewards, discount):
    discounted_rewards = list(rewards)
    pointer = len(rewards) - 1
    acc_discounted_sum = rewards[-1]
    while pointer > 0:
        acc_discounted_sum *= 0.95
        pointer -= 1
        discounted_rewards[pointer] += acc_discounted_sum
        acc_discounted_sum += rewards[pointer]
    return discounted_rewards

In [ ]:
def cg_solver(Ax_fun, b, k_iter=10, eps=10**(-6)):
    # Solver for Ax = b that uses conjugate gradient method
    # Ax_fun is supposed to be a function that takes x and returns Ax
    # Impementation of the solver is taken from Wikipedia article
    # Solver itself is supposed to be outside tf realm - all vector inside are np arrays
    # Note that it is assumed k_iter < dim(b)
    x = np.zeros(b.shape, dtype=np.float)
    r = b - Ax_fun(x)
    p = np.array(r)
    for k in range(k_iter):
        
        r_norm = float(np.sum(r * r))
        A_p = Ax_fun(p)
        alpha = r_norm / np.sum(p * A_p)
        x += alpha * p
        r -= alpha * A_p
        next_r_norm = np.sum(r * r)
        
        if next_r_norm < eps:
            break
            
        beta = next_r_norm / r_norm
        p *= beta
        p += r
        
    return x   

In [ ]:
class RL_Agent:
    
    def __init__(self, model_name):
        with tf.variable_scope(model_name):
            self.model_name = model_name
            self.session = tf.Session()
            
            self.input_layer = tf.placeholder(shape=[None, 4], dtype=tf.float32)
            self.dense1_layer = tf.layers.dense(self.input_layer, 
                                                units=4, use_bias=True, 
                                                activation=tf.nn.relu, name="dense1_weights"
                                               )
            
            self.dense2_layer = tf.layers.dense(self.dense1_layer, 
                                                units=2, use_bias=True, 
                                                activation=tf.nn.relu, name="dense2_weights"
                                               ) 
            
            self.prob_layer = tf.nn.softmax(self.dense2_layer)
            self.log_prob_layer = tf.log(self.prob_layer)
                        
            self.session.run(tf.global_variables_initializer())

    def model_variables(self):
        return [x for x in tf.trainable_variables() if self.model_name in x.name]
    
    def model_size(self):
        var_sizes = [tf.size(x) for x in self.model_variables()]
        return self.session.run(tf.reduce_sum(var_sizes))
    
    def variable_size(self):
        var_sizes = [tf.size(x) for x in self.model_variables()]
        return self.session.run(var_sizes)
            
    def predict(self, states):
        return self.session.run(self.prob_layer, feed_dict={self.input_layer: states})
    
    def grad_log_prob_actions(self, states, actions, rewards):
        # Return a sum of log_prob gradients weighted by discounted sum of future rewards
        # By design this function is supposed to be called once per each individual game
        action_mask = tf.one_hot(actions, depth=2, on_value=1.0, off_value=0.0, axis=-1)
        picked_log_prob_actions = tf.reduce_sum(action_mask * self.log_prob_layer, axis=1)
        weighted_log_prob_actions = picked_log_prob_actions * rewards
        grad_log_prob_actions = get_flattened_gradients(weighted_log_prob_actions, self.model_variables())
        return self.session.run(grad_log_prob_actions, feed_dict={self.input_layer: states})
    
    def fisher_vector_product(self, states, vector):
        # This function is supposed to return the product of estimated fisher information matrix and a specified vector
        # As I hope to reliably estimate this matrix, I take all states accumulated in a batch of games
        expected_log_prob = tf.reduce_sum(tf.stop_gradient(self.prob_layer) * self.log_prob_layer, axis=1)
        log_prob_grad = get_flattened_gradients(expected_log_prob, self.model_variables())
        grad_vector_product = tf.reduce_sum(log_prob_grad * vector)
        fisher_vector_product = -get_flattened_gradients(grad_vector_product, self.model_variables()) / states.shape[0]
        return self.session.run(fisher_vector_product, feed_dict={self.input_layer: states})            

In [ ]:
class TRPO_Learner:
    
    def __init__(self, rl_agent, game_env, trpo_delta, discount, batch_size):
        self.session = rl_agent.session
        self.agent = rl_agent
        self.env = game_env
        
        self.trpo_delta = trpo_delta
        self.discount = discount
        self.batch_size = batch_size
        
        self.reward_history = []
        self.played_games = 0
        
    def play_single_game(self):
        states = None
        actions = []
        rewards = []
        
        observation = self.env.reset().reshape((1, 4))
        done = False
        
        while done == False:
            if states is None:
                states = observation
            else:
                states = np.concatenate((states, observation), axis=0)
            prob_actions = self.agent.predict(observation)[0]
            action = np.random.choice(np.arange(len(prob_actions)), p=prob_actions)
            actions.append(action)
            observation, reward, done, info = self.env.step(action)
            observation = observation.reshape((1, 4))
            rewards.append(reward)
            
        self.reward_history.append(sum(rewards))
        self.played_games += 1
            
        return states, actions, rewards
    
    def play_batch(self, return_all_states=1):        
        grad_reward = np.zeros(self.agent.model_size())
        all_states = []
        
        for i in range(self.batch_size):

            states, actions, rewards = self.play_single_game()
            if return_all_states == 1:
                all_states.append(states)
            discounted_rewards = sum_discounted_rewards(rewards, self.discount)
            
            grad_reward += cartpole_model.grad_log_prob_actions(states, 
                                                                tf.constant(actions),
                                                                tf.constant(discounted_rewards)
                                                               )
            
        
#         print "Reward for game #", self.played_games, ": ", self.reward_history[-1]
        print "Average reward for batch #", self.played_games / self.batch_size, \
              ": ", sum(self.reward_history[-self.batch_size:]) / self.batch_size
        
            
        if return_all_states == 1:
            return grad_reward / self.batch_size, reduce(lambda x, y: np.concatenate((x, y), axis=0), all_states)
        else:
            return grad_reward / self.batch_size
    
    def trpo_step(self):
        
        grad_reward, obs_states = self.play_batch(return_all_states=1)
        Ax_fun = lambda x: self.agent.fisher_vector_product(obs_states, tf.constant(x, dtype=tf.float32))
        
        trpo_dir = cg_solver(Ax_fun, grad_reward)
        scaling = np.sqrt(2 * self.trpo_delta / np.sum(trpo_dir * Ax_fun(trpo_dir)))
        
        grads = unflatten_gradient(tf.constant(scaling * trpo_dir, dtype=tf.float32), self.agent.model_variables())
        for (grad, var) in zip(grads, self.agent.model_variables()):
            self.session.run(tf.assign_add(var, grad))
            
    def pg_step(self):
        # Policy gradient method (for debugging purposes)
        grad_reward = self.play_batch(return_all_states=0)
        grads = unflatten_gradient(tf.constant(0.1 * grad_reward, dtype=tf.float32), self.agent.model_variables())
        for (grad, var) in zip(grads, self.agent.model_variables()):
            self.session.run(tf.assign_add(var, grad))        
                    

In [ ]:
tf.reset_default_graph()

cartpole_model = RL_Agent("cartpole_model")
print cartpole_model.predict(env.reset().reshape((1, 4)))
print cartpole_model.model_variables()
print cartpole_model.model_size()

In [ ]:
trpo = TRPO_Learner(cartpole_model, env, trpo_delta=0.01, discount=0.99, batch_size=100)

# states, actions, rewards = trpo.play_single_game()

In [ ]:
for i in range(100):
    trpo.pg_step()
#     print trpo.session.run(trpo.agent.model_variables())
    
plt.plot(range(len(trpo.reward_history)), trpo.reward_history)
plt.show()

In [ ]:
print cartpole_model.grad_log_prob_actions(states, 
                                           tf.constant(actions), 
                                           tf.constant(sum_discounted_rewards(rewards, trpo.discount))
                                          )

In [ ]:
grad_reward, obs_states = trpo.play_batch()

In [ ]:
grad_reward * grad_reward

In [ ]:
trpo.agent.fisher_vector_product(obs_states, tf.constant([1.0] * trpo.agent.model_size()))

In [ ]:
A = np.array([[4, 1], [1, 3]])
Ax_fun = lambda x: np.dot(A, x)
b = np.array([1, 2], dtype=np.float).reshape((2, 1))
cg_solver(Ax_fun, b, k_iter=2)